# Tutorial: Train PyTorch Models

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [3]:
features, labels = datasets.load_breast_cancer(return_X_y=True)
x_train, x_valid, y_train, y_valid = train_test_split(features, labels, test_size=0.2)

In [4]:
features.shape, labels.shape

((569, 30), (569,))

In [5]:
import torch
import torch.nn as nn
import torch.utils.data as D
from kuma_utils.torch import TorchTrainer, EarlyStopping, TorchLogger
from kuma_utils.torch.model_zoo import TabularNet
from kuma_utils.metrics import AUC, Accuracy
from kuma_utils.utils import sigmoid

torch_xla not found.
nvidia apex not found.


In [6]:
train_ds = D.TensorDataset(
    torch.as_tensor(x_train).float(),
    torch.as_tensor(y_train.reshape(-1, 1)).float()
)
valid_ds = D.TensorDataset(
    torch.as_tensor(x_valid).float(),
    torch.as_tensor(y_valid.reshape(-1, 1)).float()
)
train_loader = D.DataLoader(train_ds, batch_size=64, shuffle=True)
valid_loader = D.DataLoader(valid_ds, batch_size=64, shuffle=False)

In [7]:
model = TabularNet(in_features=x_train.shape[1], out_features=1)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=5e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 'min', factor=0.5, patience=2, verbose=True, min_lr=1e-5)
criterion = nn.BCEWithLogitsLoss()
LogitsAcc = lambda approx, target: Accuracy().torch(sigmoid(approx), target)

fit_params = {
    'loader': train_loader,
    'loader_valid': valid_loader,
    'criterion': criterion,
    'optimizer': optimizer,
    'scheduler': scheduler,
    'num_epochs': 30,
    'callbacks': [EarlyStopping(5, target='valid_metric', maximize=True, skip_epoch=10)],
    'eval_metric': AUC().torch,
    'monitor_metrics': [LogitsAcc], 
    'logger': TorchLogger(
        'logger.log', 
        log_items=[
            'epoch', 'train_loss', 'valid_loss', 
            'valid_metric', 'valid_monitor', 'patience'], stdout=True, file=False),
    # 'calibrate_model': True, 
    'export_dir': 'results/baseline/',
    # 'resume': True
}

TorchLogger created at 20/12/01:09:42:09


In [8]:
trn = TorchTrainer(model)
trn.train(**fit_params)

09:42:39 Model is on cpu
09:42:39 [Epoch   1/ 30] train_loss=4.602656 | valid_loss=0.706972 | valid_metric=0.219805 | valid_monitor=[0.606033] 
09:42:39 [Epoch   2/ 30] train_loss=2.522258 | valid_loss=0.696433 | valid_metric=0.236039 | valid_monitor=[0.574023] 
09:42:39 [Epoch   3/ 30] train_loss=1.759894 | valid_loss=0.630515 | valid_metric=0.953571 | valid_monitor=[0.407972] 
09:42:39 [Epoch   4/ 30] train_loss=1.157770 | valid_loss=0.692414 | valid_metric=0.334091 | valid_monitor=[0.612034] 
09:42:39 [Epoch   5/ 30] train_loss=0.916939 | valid_loss=0.668145 | valid_metric=0.549675 | valid_monitor=[0.580025] 
09:42:40 [Epoch   6/ 30] train_loss=0.771432 | valid_loss=0.674632 | valid_metric=0.833442 | valid_monitor=[0.481994] 
Epoch     7: reducing learning rate of group 0 to 5.0000e-04.
09:42:40 [Epoch   7/ 30] train_loss=0.681121 | valid_loss=0.640791 | valid_metric=0.853247 | valid_monitor=[0.528009] 
09:42:40 [Epoch   8/ 30] train_loss=0.689128 | valid_loss=0.639499 | valid_metri

In [9]:
trn.outoffold[:5]

array([[ 0.5846777 ],
       [-0.8197745 ],
       [-0.03987291],
       [-0.25906694],
       [-1.9227417 ]], dtype=float32)